In [1]:
import math, random
import string

class Cromossomo():

    def __init__(self, tamanho):
        self.lista_modelos = self.criaListaModelos()
        self.tamanho = tamanho
        self.valor = []
        self.avaliacao = 0

    
    def criaListaModelos(self):##Esta sendo usado
        mod1 = list(range(0,10))
        mod2 = list(string.ascii_uppercase)
        
        #modelo = [[1..9],[A..Z]]
        modelo = [mod1, mod2]
        
        return modelo
        
    def getLetraRandom(self):##Esta sendo usado
        i = random.randint(0,25)
        
        return self.lista_modelos[1][i]
    
    def set_valor(self, novo_valor):##Esta sendo usado
        self.valor = novo_valor
    
    def inicializar(self):##Esta sendo usado
        novo_valor = []
        for i in range(self.tamanho):
            if random.random() < .5: ##Se valor.randômico < 0.50, adiciona um numero aleatório
                novo_valor.append(random.randint(0,9))
            else: novo_valor.append(self.getLetraRandom())
                
        self.set_valor(novo_valor)

    def crossover(self, outro_cromossomo):
        split_index = int(random.random() * self.tamanho)
        novo_valor = ""
        if random.random() > .5:
            novo_valor = self.valor[0:split_index] + outro_cromossomo.valor[split_index:len(outro_cromossomo.valor)]
        else:
            novo_valor = outro_cromossomo.valor[0:split_index] + self.valor[split_index:len(outro_cromossomo.valor)]
        novo_cromossomo = Cromossomo(self.tamanho)
        novo_cromossomo.set_valor(novo_valor)
        return novo_cromossomo

    def mutacao(self, chance_mutacao):
        inicio, aux, fim = ['','','']
        for i in range(self.tamanho):
            if random.random() < chance_mutacao:
                inicio = self.valor[0:i]
                fim = self.valor[i+1:self.tamanho]
                aux = self.valor[i]
                if aux == '1': aux = '0'
                else: aux = '1'
                self.set_valor(inicio+aux+fim)

    def valor_real(self, inf = 0, sup = 100):
        return inf + (sup - inf)/(2**self.tamanho - 1)*int(self.valor, 2)

    def avaliar(self, modelo):##Esta sendo usado
        for i in range(len(modelo)):
            if type(self.valor[i]) == type(modelo[i]):
                self.avaliacao += 1

        return self.avaliacao

    def __repr__(self):

        return "cromossomo:[%s] avaliacao[%.2f]" % (self.valor, self.avaliacao)

In [5]:
import math, random
import string
import re

class GATest():

    def __init__(self, tamanho_populacao, geracoes):
        self.populacao = []
        self.tam_populacao = tamanho_populacao
        self.geracoes = geracoes
        self.soma_avaliacoes = 0
    
    
    def inicializar_populacao(self, tam_cromossomo):
        for i in range(self.tam_populacao):
            self.populacao.append(Cromossomo(tam_cromossomo))
        for cromossomo in self.populacao: cromossomo.inicializar()
            
    def avaliar_populacao(self,modelo):
        for cromossomo in self.populacao:
            self.soma_avaliacoes += cromossomo.avaliar(modelo)
            print(cromossomo.valor, cromossomo.avaliacao)
    
    def executar(self):
        print("Digite seu modelo:")
        input_model = input()
        input_model_split = input_model.split()
        model = []
        for i in input_model_split:
            if re.match('^\d+$',i): model.append(int(i))
            else: model.append(i)
        
        tam_model = len(model)
        self.inicializar_populacao(tam_model)
        self.avaliar_populacao(model)

ga = GATest(4,5)
ga.executar()

Digite seu modelo:
A Z B 2
['W', 0, 1, 6] 2
['B', 6, 'K', 'C'] 2
[3, 2, 5, 'P'] 0
[8, 3, 1, 'E'] 0
